In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

DATA_ROOT = "/home/sl1471/public/"
target_path = "/home/sl1471/workspace/experiments/BX/"
data_path = DATA_ROOT + "bookcrossing/"

user_core = 20
item_core = 10

### Rating Data

In [2]:
import pandas as pd
df = pd.read_table(data_path + "BX-Book-Ratings.csv", sep=";", encoding_errors='ignore', header = 0, names = ["UserID", "ItemID", "Response"])

In [3]:
df['Timestamp'] = [i for i in range(len(df))]
df[:5]

,UserID,ItemID,Response,Timestamp
0,276725,034545104X,0,0
1,276726,0155061224,5,1
2,276727,0446520802,0,2
3,276729,052165615X,3,3
4,276729,0521795028,6,4


In [4]:
n_user, n_item = len(df.UserID.unique()), len(df.ItemID.unique())
print(f"#user: {n_user}")
print(f"#item: {n_item}")
print(f"sparsity: {1.0 - len(df) / (n_user * n_item)}")

#user: 105283
#item: 340553
sparsity: 0.9999679320064145


In [5]:
from data.preprocess import run_multicore_asymetric
multicore_data = run_multicore_asymetric(df, user_core, item_core)
# multicore_data = run_multicore_asymetric(multicore_data, 20, 10)

N-core is set to [5,100]
Filtering (20,10)-core data
Iteration 1


100%|██████████| 1149780/1149780 [00:01<00:00, 625684.14it/s]


Number of removed record: 773964
Iteration 2


100%|██████████| 375816/375816 [00:00<00:00, 767494.60it/s]


Number of removed record: 48678
Iteration 3


...

Number of removed record: 54
Iteration 7


100%|██████████| 317262/317262 [00:00<00:00, 761470.05it/s]


Number of removed record: 12
Iteration 8


100%|██████████| 317250/317250 [00:00<00:00, 654169.20it/s]


Number of removed record: 0
Size change: 1149780 --> 317250


In [6]:
n_user, n_item = len(multicore_data.UserID.unique()), len(multicore_data.ItemID.unique())
print(f"#user: {n_user}")
print(f"#item: {n_item}")
print(f"#record: {len(multicore_data)}")
print(f"sparsity: {1.0 - len(multicore_data) / (n_user * n_item)}")

#user: 3578
#item: 12048
#record: 317250
sparsity: 0.9926405334503239


In [7]:
import pandas as pd
item_meta = pd.read_table(data_path + "BX-Books.csv", sep=";", encoding_errors='ignore', error_bad_lines=False,
                          header = 0, usecols = [0,3,4], names = ["ItemID", "Year", "Publisher"])

/home/sl1471/anaconda3/envs/bmrl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/home/sl1471/anaconda3/envs/bmrl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
def filter_items(item_meta, items):
    selected_rows = [False] * len(item_meta)
    all_meta_items = item_meta["ItemID"].unique()
    for i,uid in enumerate(all_meta_items):
        if uid in items:
            selected_rows[i] = True
    return item_meta[selected_rows]
print(len(item_meta))
item_meta = filter_items(item_meta, {uid: i for i,uid in enumerate(multicore_data["ItemID"].unique())})
print("-->",len(item_meta))

271379
--> 11752


In [9]:
selected_rows = [True] * len(multicore_data)
filtered_items = {iid: i for i,iid in enumerate(item_meta["ItemID"])}
for i,iid in enumerate(multicore_data["ItemID"]):
    if iid not in filtered_items:
        selected_rows[i] = False
multicore_data = multicore_data[selected_rows]
multicore_data = run_multicore_asymetric(multicore_data, user_core, item_core)

N-core is set to [5,100]
Filtering (20,10)-core data
Iteration 1


100%|██████████| 312353/312353 [00:00<00:00, 700525.27it/s]


Number of removed record: 1451
Iteration 2


100%|██████████| 310902/310902 [00:00<00:00, 1002950.74it/s]


Number of removed record: 474
Iteration 3


100%|██████████| 310428/310428 [00:00<00:00, 1020834.43it/s]


Number of removed record: 66
Iteration 4


100%|██████████| 310362/310362 [00:00<00:00, 955807.58it/s]


Number of removed record: 0
Size change: 312353 --> 310362


In [10]:
n_user, n_item = len(multicore_data.UserID.unique()), len(multicore_data.ItemID.unique())
print(f"#user: {n_user}")
print(f"#item: {n_item}")
print(f"#record: {len(multicore_data)}")
print(f"sparsity: {1.0 - len(multicore_data) / (n_user * n_item)}")

#user: 3500
#item: 11680
#record: 310362
sparsity: 0.9924079745596869


In [11]:
from utils import set_random_seed
set_random_seed(9)
from data.preprocess import holdout_data_sequential, recheck_exist
trainset, valset, testset = holdout_data_sequential(multicore_data, holdout_type = "warm", ratio = [0.8,0.1,0.1])
trainset = trainset.reset_index(drop = True)
valset = valset.reset_index(drop = True)
testset = testset.reset_index(drop = True)
# recheck if there is any unseen item in val or test, if there is move corresponding user history into train
trainset, valset, testset = recheck_exist(trainset, valset, testset, field_name = "ItemID")

Build user history


310362it [00:00, 376309.12it/s]


Holdout user histories


100%|██████████| 3500/3500 [00:00<00:00, 6920.00it/s]


Move unseen ItemID from val to train
Moving user data
Before moving: Target DataFrame: 251210, Source Data Frame: 29576


...


#user moved: 1060
After moving: Target DataFrame: 264519, Source Data Frame: 16267
Move unseen ItemID from test to train, this may also move users in val to train
Val --> Train
Moving user data
Before moving: Target DataFrame: 264519, Source Data Frame: 16267


100%|██████████| 1352/1352 [00:00<00:00, 5474.48it/s]



#user moved: 1948
After moving: Target DataFrame: 275824, Source Data Frame: 4962
Test --> Train
Moving user data
Before moving: Target DataFrame: 275824, Source Data Frame: 29576


100%|██████████| 1552/1552 [00:00<00:00, 4307.08it/s]


#user moved: 1948
After moving: Target DataFrame: 299653, Source Data Frame: 5747


In [12]:
from utils import setup_path
save_path = target_path + "tsv_data/"
setup_path(save_path, is_dir = True)
trainset.to_csv(save_path + "train.tsv", sep = '\t', index = False)
valset.to_csv(save_path + "val.tsv", sep = '\t', index = False)
testset.to_csv(save_path + "test.tsv", sep = '\t', index = False)

error when creating ""
dir "/home" existed
dir "/home/sl1471" existed
dir "/home/sl1471/workspace" existed
dir "/home/sl1471/workspace/experiments" existed
dir "/home/sl1471/workspace/experiments/BX" existed
dir "/home/sl1471/workspace/experiments/BX/tsv_data" existed
dir "/home/sl1471/workspace/experiments/BX/tsv_data/" existed


In [13]:
len(trainset["UserID"].unique()), len(valset["UserID"].unique()), len(testset["UserID"].unique())

(3500, 1352, 1552)

In [14]:
len(trainset["ItemID"].unique()), len(valset["ItemID"].unique()), len(testset["ItemID"].unique())

(11680, 2113, 1961)

In [15]:
len(trainset), len(valset), len(testset)

(299653, 4962, 5747)

## User Meta

In [16]:
import pandas as pd
user_meta = pd.read_table(data_path + "BX-Users.csv", sep=";", encoding_errors='ignore', header = 0, names = ["UserID", "Location", "Age"])

In [17]:
import numpy as np
user_meta = user_meta.fillna(-1)
user_meta.replace(np.nan, 0)
user_meta[:5]

,UserID,Location,Age
0,1,"nyc, new york, usa",-1.0
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",-1.0
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",-1.0


In [18]:
import numpy as np
age_group = [int(age) // 20 if 0<age<60 else 6 if age>60 else -1 for age in user_meta["Age"]]
user_meta = user_meta.drop(["Age"], axis = 1)
user_meta.insert(2, "AgeGroup", age_group)
user_meta[:5]

,UserID,Location,AgeGroup
0,1,"nyc, new york, usa",-1
1,2,"stockton, california, usa",0
2,3,"moscow, yukon territory, russia",-1
3,4,"porto, v.n.gaia, portugal",0
4,5,"farnborough, hants, united kingdom",-1


In [19]:
locations = [loc.replace(' ','').split(',')[-1] for loc in user_meta["Location"]]
user_meta = user_meta.drop(["Location"], axis = 1)
user_meta.insert(1, "Location", locations)
user_meta[:5]

,UserID,Location,AgeGroup
0,1,usa,-1
1,2,usa,0
2,3,russia,-1
3,4,portugal,0
4,5,unitedkingdom,-1


In [20]:
def filter_users(user_meta, users):
    selected_rows = [False] * len(user_meta)
    all_meta_users = user_meta["UserID"].unique()
    for i,uid in enumerate(all_meta_users):
        if uid in users:
            selected_rows[i] = True
    return user_meta[selected_rows]

In [21]:
print(len(user_meta))
user_meta = filter_users(user_meta, {uid: i for i,uid in enumerate(multicore_data["UserID"].unique())})
print("-->",len(user_meta))

278858
--> 3500


In [22]:
from utils import setup_path
save_path = target_path + "meta_data/user.meta"
setup_path(save_path, is_dir = False)
user_meta.to_csv(save_path, sep = '\t', index = False)

error when creating ""
dir "/home" existed
dir "/home/sl1471" existed
dir "/home/sl1471/workspace" existed
dir "/home/sl1471/workspace/experiments" existed
dir "/home/sl1471/workspace/experiments/BX" existed
dir "/home/sl1471/workspace/experiments/BX/meta_data" existed


In [23]:
from data.preprocess import build_vocab
from utils import setup_path
save_path = target_path + "meta_data/user_fields.vocab"
setup_path(save_path, is_dir = False)
build_vocab(user_meta, save_path, ["UserID", "Location", "AgeGroup"])

error when creating ""
dir "/home" existed
dir "/home/sl1471" existed
dir "/home/sl1471/workspace" existed
dir "/home/sl1471/workspace/experiments" existed
dir "/home/sl1471/workspace/experiments/BX" existed
dir "/home/sl1471/workspace/experiments/BX/meta_data" existed
Vocab file saved to: /home/sl1471/workspace/experiments/BX/meta_data/user_fields.vocab


In [24]:
import pandas as pd
user_fields_meta = pd.DataFrame({"field_name": ["UserID", "Location", "AgeGroup"], 
                                 "field_type": ["nominal", "nominal", "nominal"], 
                                 "value_type": ["int", "str", "int"], 
                                 "field_enc": ["v2id", "v2onehot", "v2id"]})
user_fields_meta.to_csv(target_path + "meta_data/user_fields.meta", 
                        sep = '\t', index = False)

## Item Features

In [25]:
import pandas as pd
item_meta = pd.read_table(data_path + "BX-Books.csv", sep=";", encoding_errors='ignore', error_bad_lines=False,
                          header = 0, usecols = [0,3,4], names = ["ItemID", "Year", "Publisher"])

/home/sl1471/anaconda3/envs/bmrl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/home/sl1471/anaconda3/envs/bmrl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [26]:
item_meta[:5]

,ItemID,Year,Publisher
0,0195153448,2002,Oxford University Press
1,0002005018,2001,HarperFlamingo Canada
2,0060973129,1991,HarperPerennial
3,0374157065,1999,Farrar Straus Giroux
4,0393045218,1999,W. W. Norton &amp; Company


In [27]:
item_meta = filter_items(item_meta, {iid: i for i,iid in enumerate(multicore_data["ItemID"].unique())})
print(len(item_meta), len(multicore_data["ItemID"].unique()))

11680 11680


In [28]:
for i,row in enumerate(item_meta["Year"]):
    if "ROBERT" in row:
        print(item_meta.iloc[i])
        item_meta["Publisher"].iloc[i], item_meta["Year"].iloc[i] = item_meta["Year"].iloc[i], item_meta["Publisher"].iloc[i]
        print(item_meta.iloc[i])
        break

In [29]:
from utils import setup_path
save_path = target_path + "meta_data/item.meta"
setup_path(save_path, is_dir = False)
item_meta.to_csv(save_path, sep = '\t', index = False)

error when creating ""
dir "/home" existed
dir "/home/sl1471" existed
dir "/home/sl1471/workspace" existed
dir "/home/sl1471/workspace/experiments" existed
dir "/home/sl1471/workspace/experiments/BX" existed
dir "/home/sl1471/workspace/experiments/BX/meta_data" existed


In [30]:
from data.preprocess import build_vocab
from utils import setup_path
save_path = target_path + "meta_data/item_fields.vocab"
setup_path(save_path, is_dir = False)
build_vocab(item_meta, save_path, ["ItemID", "Year", "Publisher"])

error when creating ""
dir "/home" existed
dir "/home/sl1471" existed
dir "/home/sl1471/workspace" existed
dir "/home/sl1471/workspace/experiments" existed
dir "/home/sl1471/workspace/experiments/BX" existed
dir "/home/sl1471/workspace/experiments/BX/meta_data" existed
Vocab file saved to: /home/sl1471/workspace/experiments/BX/meta_data/item_fields.vocab


In [31]:
import pandas as pd
item_fields_meta = pd.DataFrame({"field_name": ["ItemID", "Year", "Publisher"], 
                                 "field_type": ["nominal", "ordinal", "nominal"], 
                                 "value_type": ["str", "int", "str"], 
                                 "field_enc": ["v2id", "v2id", "v2id"]})
item_fields_meta.to_csv(target_path + "meta_data/item_fields.meta", 
                        sep = '\t', index = False)